In [20]:
import drjit as dr
import mitsuba as mi

mi.set_variant('cuda_ad_rgb')


In [21]:
scene = mi.load_file('../scenes/diff_sphere.xml')
params = mi.traverse(scene)

key = "diffsphere.to_world"
# Our latent variables
pos = mi.Float([0.2, 0.2])
dr.enable_grad(pos)

# The wall color now depends on `theta`
params[key] = mi.Transform4f.translate([pos[0], pos[1], 0.0]).scale(0.5)

# Propagate this change to the scene internal state
params.update()

image = mi.render(scene, params, spp=128)
mi.Bitmap(image)

Bitmap[
  pixel_format = rgb,
  component_format = float32,
  size = [16, 16],
  srgb_gamma = 0,
  struct = Struct<12>[
    float32 R; // @0, premultiplied alpha
    float32 G; // @4, premultiplied alpha
    float32 B; // @8, premultiplied alpha
  ],
  data = [ 3 KiB of image data ]
]

In [14]:
def get_forward_grad(flag):
    scene = mi.load_file('../scenes/diff_sphere.xml')
    params = mi.traverse(scene)

    key = "diffsphere.to_world"
    # Our latent variables
    x, y = mi.Float(0.5), mi.Float(0.5)
    dr.enable_grad(x, y)

    # The wall color now depends on `theta`
    params[key] = mi.Transform4f.translate([x, y, 0.0]).scale(0.5)

    dr.forward(pos, flag | dr.ADFlag.ClearEdges)
    image = mi.render(scene, params, spp=128)

    # Forward-propagate the gradients to the image
    dr.forward_to(image)

    # Visualize the gradient image
    return dr.grad(image)

In [11]:
grad = get_forward_grad(dr.ADFlag.BackPropGrad)
print(grad.max_(), grad.min_())
mi.Bitmap(grad / (grad.max_() - grad.min_()))

TypeError: forward_to(): the argument does not depend on the input variable(s) being differentiated. Raising an exception since this is usually indicative of a bug (for example, you may have forgotten to call dr.enable_grad(..)). If this is expected behavior, skip the call to forward_to(..) if ek.grad_enabled(..) returns False.

In [ ]:
sq_grad_sum = get_forward_grad(dr.ADFlag.BackPropVar)
print(sq_grad_sum.max_(), sq_grad_sum.min_())
mi.Bitmap(sq_grad_sum / (sq_grad_sum.max_() - sq_grad_sum.min_()))

TypeError: forward_to(): the argument does not depend on the input variable(s) being differentiated. Raising an exception since this is usually indicative of a bug (for example, you may have forgotten to call dr.enable_grad(..)). If this is expected behavior, skip the call to forward_to(..) if ek.grad_enabled(..) returns False.

In [ ]:
ones = get_forward_grad(dr.ADFlag.BackPropOnes)
print(ones.max_(), ones.min_())
mi.Bitmap(ones / (ones.max_() - ones.min_()))

[2.9248955249786377] [0.0]


Bitmap[
  pixel_format = rgb,
  component_format = float32,
  size = [256, 256],
  srgb_gamma = 0,
  struct = Struct<12>[
    float32 R; // @0, premultiplied alpha
    float32 G; // @4, premultiplied alpha
    float32 B; // @8, premultiplied alpha
  ],
  data = [ 768 KiB of image data ]
]

In [ ]:
var = sq_grad_sum - grad * grad / ones
print(var.max_(), var.min_())
mi.Bitmap(var / (var.max_() - var.min_()))

[5.039287316321861e-07] [-4.749757067656901e-07]


Bitmap[
  pixel_format = rgb,
  component_format = float32,
  size = [256, 256],
  srgb_gamma = 0,
  struct = Struct<12>[
    float32 R; // @0, premultiplied alpha
    float32 G; // @4, premultiplied alpha
    float32 B; // @8, premultiplied alpha
  ],
  data = [ 768 KiB of image data ]
]

In [15]:
def get_backward_grad(flag):
    scene = mi.load_file('../scenes/diff_sphere.xml')
    params = mi.traverse(scene)

    key = "diffsphere.to_world"
    # Our latent variables
    x, y = mi.Float(0.2), mi.Float(0.2)
    dr.enable_grad(x, y)

    # The wall color now depends on `theta`
    params[key] = mi.Transform4f.translate([x, y, 0.0]).scale(0.5)

    # Propagate this change to the scene internal state
    params.update()

    image = mi.render(scene, params, spp=512)

    # Backward-propagate the gradients
    dr.backward(image, flag | dr.ADFlag.ClearEdges)

    return dr.grad([x, y])

In [16]:
grad_b = get_backward_grad(dr.ADFlag.BackPropGrad)
print(grad_b)

[[-21.624645233154297], [-16.840621948242188]]


In [17]:
sq_grad_sum_b = get_backward_grad(dr.ADFlag.BackPropVar)
print(sq_grad_sum_b)

[[-21.624094009399414], [-16.975772857666016]]


In [18]:
ones_b = get_backward_grad(dr.ADFlag.BackPropOnes)
print(ones_b)

[[-21.624788284301758], [-16.929744720458984]]


In [19]:
var_b = sq_grad_sum_b - grad_b * grad_b / ones_b
print(var_b)

TypeError: can't multiply sequence by non-int of type 'list'